选取数据集Trending YouTube Video Statistics中的 youtube_USvideos使用Apriori算法进行关联规则挖掘

数据处理：对数据集进行处理，转换成适合关联规则挖掘的形式

In [10]:
from numpy import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
color=sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm,skew
pd.set_option('display.float_format',lambda x:'{:.3f}'.format(x))

df1=pd.read_csv('./youtube-new/USvideos.csv')
 
def loadDataSet():
    clean_data = pd.DataFrame(pd.read_csv("./youtube-new/USvideos.csv"))
    new_data = []
    # 关联规则挖掘数据处理
    for v in range(1, len(clean_data)):
        item = []
        item.append("category_id" + '='+ str(clean_data.loc[v,'category_id']))
        item.append("comments_disabled" + '='+ str(clean_data.loc[v,'comments_disabled']))
        item.append("ratings_disabled" + '='+ str(clean_data.loc[v,'ratings_disabled']))
        item.append("video_error_or_removed" + '='+ str(clean_data.loc[v,'video_error_or_removed']))
        #print(item)
        new_data.append(item)
    print(len(new_data))
    return new_data

找出频繁项集，满足最小支持度阈值的所有项集，最小支持度设置为0.5

获取候选1项集，dataSet为事务集。返回一个list，每个元素都是set集合

In [11]:
def createC1(dataSet):
    C1 = []  
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort() 
    return list(map(frozenset, C1))  


def scanD(dataSet, Ck, minSupport):
    ssCnt = {}   
    for tid in dataSet:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1   
    numItems = float(len(dataSet))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)  
        supportData[key] = support
    return retList, supportData 

生成候选项集

In [12]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]   
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

# 获取事务集中的所有的频繁项集

def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)  
    D = list(map(set, dataSet))  
    L1, supportData = scanD(D, C1, minSupport)  
    L = [L1]  
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)  
        Lk, supK = scanD(D, Ck, minSupport)  
        L.append(Lk);supportData.update(supK)  
        k += 1
    return L, supportData

生成关联规则

In [13]:
if __name__=='__main__':
    dataSet = loadDataSet()  
    L, suppData = apriori(dataSet,minSupport=0.7)
    print(L,suppData)


40948
[[frozenset({'video_error_or_removed=False'}), frozenset({'ratings_disabled=False'}), frozenset({'comments_disabled=False'})], [frozenset({'ratings_disabled=False', 'comments_disabled=False'}), frozenset({'video_error_or_removed=False', 'comments_disabled=False'}), frozenset({'video_error_or_removed=False', 'ratings_disabled=False'})], [frozenset({'video_error_or_removed=False', 'comments_disabled=False', 'ratings_disabled=False'})], []] {frozenset({'category_id=24'}): 0.24333300771710462, frozenset({'comments_disabled=False'}): 0.984541369541858, frozenset({'ratings_disabled=False'}): 0.9958728143010648, frozenset({'video_error_or_removed=False'}): 0.9994383120054704, frozenset({'category_id=23'}): 0.08442414769952135, frozenset({'category_id=28'}): 0.058635342385464494, frozenset({'category_id=1'}): 0.05726775422487057, frozenset({'category_id=25'}): 0.060735567060662306, frozenset({'category_id=17'}): 0.053091726091628405, frozenset({'category_id=10'}): 0.15805411741721206, fr

关联规则，其支持度和置信度结果如下

In [14]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 26 16:31:10 2019

@author: admin
"""

from numpy import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
color=sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm,skew
pd.set_option('display.float_format',lambda x:'{:.3f}'.format(x))

df1=pd.read_csv('./youtube-new/USvideos.csv')
 
def loadDataSet():
    clean_data = pd.DataFrame(pd.read_csv("./youtube-new/USvideos.csv"))
    new_data = []
    # 关联规则挖掘数据处理
    for v in range(1, len(clean_data)):
        item = []
        item.append("category_id" + '='+ str(clean_data.loc[v,'category_id']))
        item.append("comments_disabled" + '='+ str(clean_data.loc[v,'comments_disabled']))
        item.append("ratings_disabled" + '='+ str(clean_data.loc[v,'ratings_disabled']))
        item.append("video_error_or_removed" + '='+ str(clean_data.loc[v,'video_error_or_removed']))
        #print(item)
        new_data.append(item)
    print(len(new_data))
    return new_data

 
# 获取候选1项集，dataSet为事务集。返回一个list，每个元素都是set集合
def createC1(dataSet):
    C1 = []   # 元素个数为1的项集（非频繁项集，因为还没有同最小支持度比较）
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()  # 这里排序是为了，生成新的候选集时可以直接认为两个n项候选集前面的部分相同
    # 因为除了候选1项集外其他的候选n项集都是以二维列表的形式存在，所以要将候选1项集的每一个元素都转化为一个单独的集合。
    return list(map(frozenset, C1))   #map(frozenset, C1)的语义是将C1由Python列表转换为不变集合（frozenset，Python中的数据结构）




# 找出候选集中的频繁项集
# dataSet为全部数据集，Ck为大小为k（包含k个元素）的候选项集，minSupport为设定的最小支持度
def scanD(dataSet, Ck, minSupport):
    ssCnt = {}   # 记录每个候选项的个数
    for tid in dataSet:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1   # 计算每一个项集出现的频率
    numItems = float(len(dataSet))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)  #将频繁项集插入返回列表的首部
        supportData[key] = support
    return retList, supportData   #retList为在Ck中找出的频繁项集（支持度大于minSupport的），supportData记录各频繁项集的支持度


# 通过频繁项集列表Lk和项集个数k生成候选项集C(k+1)。
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            # 前k-1项相同时，才将两个集合合并，合并后才能生成k+1项
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]   # 取出两个集合的前k-1个元素
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

# 获取事务集中的所有的频繁项集
# Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)  # 从事务集中获取候选1项集
    D = list(map(set, dataSet))  # 将事务集的每个元素转化为集合
    L1, supportData = scanD(D, C1, minSupport)  # 获取频繁1项集和对应的支持度
    L = [L1]  # L用来存储所有的频繁项集
    k = 2
    while (len(L[k-2]) > 0): # 一直迭代到项集数目过大而在事务集中不存在这种n项集
        Ck = aprioriGen(L[k-2], k)   # 根据频繁项集生成新的候选项集。Ck表示项数为k的候选项集
        Lk, supK = scanD(D, Ck, minSupport)  # Lk表示项数为k的频繁项集，supK为其支持度
        L.append(Lk);supportData.update(supK)  # 添加新频繁项集和他们的支持度
        k += 1
    return L, supportData



if __name__=='__main__':
    dataSet = loadDataSet()  # 获取事务集。每个元素都是列表
    # C1 = createC1(dataSet)  # 获取候选1项集。每个元素都是集合
    # D = list(map(set, dataSet))  # 转化事务集的形式，每个元素都转化为集合。
    # L1, suppDat = scanD(D, C1, 0.5)
    # print(L1,suppDat)


    L, suppData = apriori(dataSet,minSupport=0.7)
    print(L,suppData)


40948
[[frozenset({'video_error_or_removed=False'}), frozenset({'ratings_disabled=False'}), frozenset({'comments_disabled=False'})], [frozenset({'ratings_disabled=False', 'comments_disabled=False'}), frozenset({'video_error_or_removed=False', 'comments_disabled=False'}), frozenset({'video_error_or_removed=False', 'ratings_disabled=False'})], [frozenset({'video_error_or_removed=False', 'comments_disabled=False', 'ratings_disabled=False'})], []] {frozenset({'category_id=24'}): 0.24333300771710462, frozenset({'comments_disabled=False'}): 0.984541369541858, frozenset({'ratings_disabled=False'}): 0.9958728143010648, frozenset({'video_error_or_removed=False'}): 0.9994383120054704, frozenset({'category_id=23'}): 0.08442414769952135, frozenset({'category_id=28'}): 0.058635342385464494, frozenset({'category_id=1'}): 0.05726775422487057, frozenset({'category_id=25'}): 0.060735567060662306, frozenset({'category_id=17'}): 0.053091726091628405, frozenset({'category_id=10'}): 0.15805411741721206, fr

In [15]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 26 16:31:10 2019

@author: admin
"""

from numpy import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
color=sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from scipy.stats import norm,skew
pd.set_option('display.float_format',lambda x:'{:.3f}'.format(x))

df1=pd.read_csv('./youtube-new/USvideos.csv')
 
def loadDataSet():
    clean_data = pd.DataFrame(pd.read_csv("./youtube-new/USvideos.csv"))
    new_data = []
    # 关联规则挖掘数据处理
    for v in range(1, len(clean_data)):
        item = []
        item.append("category_id" + '='+ str(clean_data.loc[v,'category_id']))
        item.append("comments_disabled" + '='+ str(clean_data.loc[v,'comments_disabled']))
        item.append("ratings_disabled" + '='+ str(clean_data.loc[v,'ratings_disabled']))
        item.append("video_error_or_removed" + '='+ str(clean_data.loc[v,'video_error_or_removed']))
        #print(item)
        new_data.append(item)
    print(len(new_data))
    return new_data

 
# 获取候选1项集，dataSet为事务集。返回一个list，每个元素都是set集合
def createC1(dataSet):
    C1 = []   # 元素个数为1的项集（非频繁项集，因为还没有同最小支持度比较）
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()  # 这里排序是为了，生成新的候选集时可以直接认为两个n项候选集前面的部分相同
    # 因为除了候选1项集外其他的候选n项集都是以二维列表的形式存在，所以要将候选1项集的每一个元素都转化为一个单独的集合。
    return list(map(frozenset, C1))   #map(frozenset, C1)的语义是将C1由Python列表转换为不变集合（frozenset，Python中的数据结构）




# 找出候选集中的频繁项集
# dataSet为全部数据集，Ck为大小为k（包含k个元素）的候选项集，minSupport为设定的最小支持度
def scanD(dataSet, Ck, minSupport):
    ssCnt = {}   # 记录每个候选项的个数
    for tid in dataSet:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1   # 计算每一个项集出现的频率
    numItems = float(len(dataSet))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)  #将频繁项集插入返回列表的首部
        supportData[key] = support
    return retList, supportData   #retList为在Ck中找出的频繁项集（支持度大于minSupport的），supportData记录各频繁项集的支持度


# 通过频繁项集列表Lk和项集个数k生成候选项集C(k+1)。
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            # 前k-1项相同时，才将两个集合合并，合并后才能生成k+1项
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]   # 取出两个集合的前k-1个元素
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

# 获取事务集中的所有的频繁项集
# Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
def apriori(dataSet, minSupport=0.5):
    C1 = createC1(dataSet)  # 从事务集中获取候选1项集
    D = list(map(set, dataSet))  # 将事务集的每个元素转化为集合
    L1, supportData = scanD(D, C1, minSupport)  # 获取频繁1项集和对应的支持度
    L = [L1]  # L用来存储所有的频繁项集
    k = 2
    while (len(L[k-2]) > 0): # 一直迭代到项集数目过大而在事务集中不存在这种n项集
        Ck = aprioriGen(L[k-2], k)   # 根据频繁项集生成新的候选项集。Ck表示项数为k的候选项集
        Lk, supK = scanD(D, Ck, minSupport)  # Lk表示项数为k的频繁项集，supK为其支持度
        L.append(Lk);supportData.update(supK)  # 添加新频繁项集和他们的支持度
        k += 1
    return L, supportData



if __name__=='__main__':
    dataSet = loadDataSet()  # 获取事务集。每个元素都是列表
    # C1 = createC1(dataSet)  # 获取候选1项集。每个元素都是集合
    # D = list(map(set, dataSet))  # 转化事务集的形式，每个元素都转化为集合。
    # L1, suppDat = scanD(D, C1, 0.5)
    # print(L1,suppDat)


    L, suppData = apriori(dataSet,minSupport=0.3)
    print(L,suppData)

40948
[[frozenset({'video_error_or_removed=False'}), frozenset({'ratings_disabled=False'}), frozenset({'comments_disabled=False'})], [frozenset({'ratings_disabled=False', 'comments_disabled=False'}), frozenset({'video_error_or_removed=False', 'comments_disabled=False'}), frozenset({'video_error_or_removed=False', 'ratings_disabled=False'})], [frozenset({'video_error_or_removed=False', 'comments_disabled=False', 'ratings_disabled=False'})], []] {frozenset({'category_id=24'}): 0.24333300771710462, frozenset({'comments_disabled=False'}): 0.984541369541858, frozenset({'ratings_disabled=False'}): 0.9958728143010648, frozenset({'video_error_or_removed=False'}): 0.9994383120054704, frozenset({'category_id=23'}): 0.08442414769952135, frozenset({'category_id=28'}): 0.058635342385464494, frozenset({'category_id=1'}): 0.05726775422487057, frozenset({'category_id=25'}): 0.060735567060662306, frozenset({'category_id=17'}): 0.053091726091628405, frozenset({'category_id=10'}): 0.15805411741721206, fr

可以看出，一旦降低可信度阈值，就可以获得更多的规则。
修改过后只有支持度不小于 0.3 的项集被选中到 L 中作为频繁项集，根据不同的需求，我们可以设定最小支持度的值，从而得到我们想要的频繁项集。

对规则进行提升度（lift）评价

In [16]:
from numpy import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
color=sns.color_palette()
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm,skew
pd.set_option('display.float_format',lambda x:'{:.3f}'.format(x))

 

def loadDataSet():
    clean_data = pd.DataFrame(pd.read_csv("./youtube-new/USvideos.csv"))
    new_data = []
    # 关联规则挖掘数据处理
    for v in range(1, len(clean_data)):
        item = []
        item.append("category_id" + '='+ str(clean_data.loc[v,'category_id']))
        item.append("comments_disabled" + '='+ str(clean_data.loc[v,'comments_disabled']))
        item.append("ratings_disabled" + '='+ str(clean_data.loc[v,'ratings_disabled']))
        item.append("video_error_or_removed" + '='+ str(clean_data.loc[v,'video_error_or_removed']))
        #print(item)
        new_data.append(item)
    print(len(new_data))
    return new_data

 
 
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # 映射为frozenset唯一性的，可使用其构造字典
    return list(map(frozenset, C1))      
 
 
# 从候选K项集到频繁K项集（支持度计算）
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
            supportData[key] = support  
    return retList, supportData
 
 
def calSupport(D, Ck, min_support):
    dict_sup = {}
    for i in D:
        for j in Ck:
            if j.issubset(i):
                if not j in dict_sup:
                    dict_sup[j] = 1
                else:
                    dict_sup[j] += 1
    sumCount = float(len(D))
    supportData = {}
    relist = []
    for i in dict_sup:
        temp_sup = dict_sup[i] / sumCount
        if temp_sup >= min_support:
            relist.append(i)
            supportData[i] = temp_sup  # 此处可设置返回全部的支持度数据（或者频繁项集的支持度数据）
    return relist, supportData
 
 
# 改进剪枝算法
def aprioriGen(Lk, k):  # 创建候选K项集 ##LK为频繁K项集
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(Lk[i])[:k - 2]
            L2 = list(Lk[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:  # 前k-1项相等，则可相乘，这样可防止重复项出现
                #  进行剪枝（a1为k项集中的一个元素，b为它的所有k-1项子集）
                a = Lk[i] | Lk[j]  # a为frozenset()集合
                a1 = list(a)
                b = []
                # 遍历取出每一个元素，转换为set，依次从a1中剔除该元素，并加入到b中
                for q in range(len(a1)):
                    t = [a1[q]]
                    tt = frozenset(set(a1) - set(t))
                    b.append(tt)
                t = 0
                for w in b:
                    # 当b（即所有k-1项子集）都是Lk（频繁的）的子集，则保留，否则删除。
                    if w in Lk:
                        t += 1
                if t == len(b):
                    retList.append(b[0] | b[1])
    return retList
 
 
def apriori(dataSet, minSupport=0.2):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))  # 使用list()转换为列表
    L1, supportData = calSupport(D, C1, minSupport)
    L = [L1]  
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minSupport)  
        supportData.update(supK)
        L.append(Lk)  
        k += 1
    del L[-1]  
    return L, supportData  
 

def getSubset(fromList, toList):
    for i in range(len(fromList)):
        t = [fromList[i]]
        tt = frozenset(set(fromList) - set(t))
        if not tt in toList:
            toList.append(tt)
            tt = list(tt)
            if len(tt) > 1:
                getSubset(tt, toList)
 
 
def calcConf(freqSet, H, supportData, ruleList, minConf=0.7):
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]  # 计算置信度
        lift = supportData[freqSet] / (supportData[conseq] * supportData[freqSet - conseq])
 
        if conf >= minConf and lift > 1:
            print(freqSet - conseq, '-->', conseq, '支持度为：', round(supportData[freqSet - conseq], 2), '置信度为：', conf,
                  'lift值为：', round(lift, 2))
            ruleList.append((freqSet - conseq, conseq, conf))
 

def gen_rule(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):  
        for freqSet in L[i]:  
            H1 = list(freqSet)
            all_subset = []
            getSubset(H1, all_subset)  
            calcConf(freqSet, all_subset, supportData, bigRuleList, minConf)
    return bigRuleList
 
 
if __name__ == '__main__':
    dataSet = loadDataSet()
    L, supportData = apriori(dataSet, minSupport=0.2)
    rule = gen_rule(L, supportData, minConf=0.7)

40948
frozenset({'ratings_disabled=False'}) --> frozenset({'comments_disabled=False'}) 支持度为： 1.0 置信度为： 0.9870766816253465 lift值为： 1.0
frozenset({'comments_disabled=False'}) --> frozenset({'ratings_disabled=False'}) 支持度为： 0.98 置信度为： 0.9984373062135682 lift值为： 1.0
frozenset({'category_id=24'}) --> frozenset({'ratings_disabled=False'}) 支持度为： 0.24 置信度为： 0.9969891609795263 lift值为： 1.0
frozenset({'comments_disabled=False', 'category_id=24'}) --> frozenset({'ratings_disabled=False'}) 支持度为： 0.24 置信度为： 0.9988738738738739 lift值为： 1.0
frozenset({'category_id=24'}) --> frozenset({'ratings_disabled=False', 'video_error_or_removed=False'}) 支持度为： 0.24 置信度为： 0.9961862705740666 lift值为： 1.0
frozenset({'video_error_or_removed=False', 'category_id=24'}) --> frozenset({'ratings_disabled=False'}) 支持度为： 0.24 置信度为： 0.9969867416633186 lift值为： 1.0
frozenset({'comments_disabled=False'}) --> frozenset({'video_error_or_removed=False', 'ratings_disabled=False'}) 支持度为： 0.98 置信度为： 0.9978667989582041 lift值为： 1.0
froze